# Ranker that can takes into accound different features

# Set up

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
import os
import sys
from typing import List

import dill
import lightning as L
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
from dotenv import load_dotenv
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import MLFlowLogger
from loguru import logger
from mlflow.models.signature import infer_signature
from pydantic import BaseModel
from torch.utils.data import DataLoader
from qdrant_client import QdrantClient

import mlflow

load_dotenv()

sys.path.insert(0, "..")

from src.data_prep_utils import chunk_transform
from src.dataset import UserItemBinaryDFDataset
from src.id_mapper import IDMapper
from src.ranker.inference import RankerInferenceWrapper
from src.ranker.model import Ranker
from src.ranker.trainer import LitRanker
from src.ann import AnnIndex
from src.viz import blueq_colors

# Controller

In [3]:
max_epochs = 100

In [46]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "RecSys MVP - Ranker"
    run_name: str = "017-item-popularity"
    notebook_persist_dp: str = None
    random_seed: int = 41
    device: str = None

    item_metadata_pipeline_fp: str = "../data/item_metadata_pipeline.dill"
    qdrant_url: str = None
    qdrant_collection_name: str = "item_desc_sbert"

    max_epochs: int = max_epochs
    batch_size: int = 128
    tfm_chunk_size: int = 10000
    neg_to_pos_ratio: int = 3

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"
    item_feature_cols: List[str] = [
        "main_category",
        "categories",
        "price",
        "parent_asin_rating_cnt_365d",
        "parent_asin_rating_avg_prev_rating_365d",
        "parent_asin_rating_cnt_90d",
        "parent_asin_rating_avg_prev_rating_90d",
        "parent_asin_rating_cnt_30d",
        "parent_asin_rating_avg_prev_rating_30d",
        "parent_asin_rating_cnt_7d",
        "parent_asin_rating_avg_prev_rating_7d",
    ]

    top_K: int = 100
    top_k: int = 10

    embedding_dim: int = 128
    dropout: float = 0.3
    early_stopping_patience: int = 5
    learning_rate: float = 0.0003
    l2_reg: float = 1e-4

    mlf_item2vec_model_name: str = "item2vec"
    mlf_model_name: str = "ranker"
    min_roc_auc: float = 0.7

    best_checkpoint_path: str = None

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        os.makedirs(self.notebook_persist_dp, exist_ok=True)

        if not (qdrant_host := os.getenv("QDRANT_HOST")):
            raise Exception(f"Environment variable QDRANT_HOST is not set.")

        qdrant_port = os.getenv("QDRANT_PORT")
        self.qdrant_url = f"{qdrant_host}:{qdrant_port}"
        
        if not (mlflow_uri := os.environ.get("MLFLOW_TRACKING_URI")):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )
            self._mlf_logger = MLFlowLogger(
                experiment_name=self.experiment_name,
                run_name=self.run_name,
                tracking_uri=mlflow_uri,
                log_model=True,
            )

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

2024-10-27 23:18:09.018 | INFO     | __main__:init:69 - Setting up MLflow experiment RecSys MVP - Ranker - run 017-item-popularity...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "RecSys MVP - Ranker",
  "run_name": "017-item-popularity",
  "notebook_persist_dp": "/Users/dvq/frostmourne/recsys-mvp/notebooks/data/017-item-popularity",
  "random_seed": 41,
  "device": null,
  "item_metadata_pipeline_fp": "../data/item_metadata_pipeline.dill",
  "qdrant_url": "localhost:6333",
  "qdrant_collection_name": "item_desc_sbert",
  "max_epochs": 100,
  "batch_size": 128,
  "tfm_chunk_size": 10000,
  "neg_to_pos_ratio": 3,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "item_feature_cols": [
    "main_category",
    "categories",
    "price",
    "parent_asin_rating_cnt_365d",
    "parent_asin_rating_avg_prev_rating_365d",
    "parent_asin_rating_cnt_90d",
    "parent_asin_rating_avg_prev_rating_90d",
    "parent_asin_rating_cnt_30d",
    "parent_asin_rating_avg_prev_rating_30d",
    "parent_asin_rating_cnt_7d",
    "parent_asin_rating_avg

# Implement

In [5]:
def init_model(
    n_users, n_items, embedding_dim, item_feature_size, dropout, item_embedding=None
):
    model = Ranker(
        n_users,
        n_items,
        embedding_dim,
        item_feature_size=item_feature_size,
        dropout=dropout,
        item_embedding=item_embedding,
    )
    return model

## Load pretrained Item2Vec embeddings

In [6]:
mlf_client = mlflow.MlflowClient()
model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{args.mlf_item2vec_model_name}@champion"
)
skipgram_model = model.unwrap_python_model().model
embedding_0 = skipgram_model.embeddings(torch.tensor(0))
embedding_dim = embedding_0.size()[0]
id_mapping = model.unwrap_python_model().id_mapping
pretrained_item_embedding = skipgram_model.embeddings

In [7]:
assert (
    pretrained_item_embedding.embedding_dim == args.embedding_dim
), "Mismatch pretrained item_embedding dimension"

## Load vectorized item features

In [8]:
with open(args.item_metadata_pipeline_fp, "rb") as f:
    item_metadata_pipeline = dill.load(f)

## Load ANN Index

In [9]:
ann_index = AnnIndex(args.qdrant_url, args.qdrant_collection_name)

In [10]:
vector = ann_index.get_vector_by_ids([0])[0]
embedding_dim = vector.shape[0]

  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
sbert_embedding_dim = vector.shape[0]
neighbors = ann_index.get_neighbors_by_ids([0])
neighbors

  0%|          | 0/1 [00:00<?, ?it/s]

[ScoredPoint(id=0, version=0, score=1.0, payload={'parent_asin': '0375869026', 'title': 'Wonder'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1916, version=59, score=0.9272537, payload={'parent_asin': 'B005GFPZYK', 'title': 'American Sniper: The Autobiography of the Most Lethal Sniper in U.S. Military History'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1968, version=61, score=0.9080587, payload={'parent_asin': 'B005ZBO4VA', 'title': 'Tekken Hybrid - Playstation 3'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=166, version=5, score=0.899632, payload={'parent_asin': 'B00005OARM', 'title': 'Golden Sun'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=3896, version=121, score=0.89472985, payload={'parent_asin': 'B07CD6F5PX', 'title': 'Dragon Quest Xi: Echoes of An Elusive Age - PlayStation 4'}, vector=None, shard_key=None, order_value=None)]

# Test implementation

In [12]:
embedding_dim = 8
batch_size = 2

# Mock data
user_indices = [0, 0, 1, 2, 2]
item_indices = [0, 1, 2, 3, 4]
timestamps = [0, 1, 2, 3, 4]
ratings = [0, 4, 5, 3, 0]
item_sequences = [
    [-1, -1, 2, 3],
    [-1, -1, 2, 3],
    [-1, -1, 1, 3],
    [-1, -1, 2, 1],
    [-1, -1, 2, 1],
]
main_category = [
    "All Electronics",
    "Video Games",
    "All Electronics",
    "Video Games",
    "Unknown",
]
categories = [[], ["Headsets"], ["Video Games"], [], ["blah blah"]]
title = ["World of Warcraft", "DotA 2", "Diablo IV", "Football Manager 2024", "Unknown"]
description = [[], [], ["Video games blah blah"], [], ["blah blah"]]
price = ["from 14.99", "14.99", "price: 9.99", "20 dollars", "None"]
parent_asin_rating_cnt_365d = [0,1,2,3,4]
parent_asin_rating_avg_prev_rating_365d = [4.0,3.5,4.5,5.0,2.0]
parent_asin_rating_cnt_90d = [0,1,2,3,4]
parent_asin_rating_avg_prev_rating_90d = [4.0,3.5,4.5,5.0,2.0]
parent_asin_rating_cnt_30d = [0,1,2,3,4]
parent_asin_rating_avg_prev_rating_30d = [4.0,3.5,4.5,5.0,2.0]
parent_asin_rating_cnt_7d = [0,1,2,3,4]
parent_asin_rating_avg_prev_rating_7d = [4.0,3.5,4.5,5.0,2.0]

train_df = pd.DataFrame(
    {
        "user_indice": user_indices,
        "item_indice": item_indices,
        args.timestamp_col: timestamps,
        args.rating_col: ratings,
        "item_sequence": item_sequences,
        "main_category": main_category,
        "title": title,
        "description": description,
        "categories": categories,
        "price": price,
        "parent_asin_rating_cnt_365d": parent_asin_rating_cnt_365d,
        "parent_asin_rating_avg_prev_rating_365d": parent_asin_rating_avg_prev_rating_365d,
        "parent_asin_rating_cnt_90d": parent_asin_rating_cnt_90d,
        "parent_asin_rating_avg_prev_rating_90d": parent_asin_rating_avg_prev_rating_90d,
        "parent_asin_rating_cnt_30d": parent_asin_rating_cnt_30d,
        "parent_asin_rating_avg_prev_rating_30d": parent_asin_rating_avg_prev_rating_30d,
        "parent_asin_rating_cnt_7d": parent_asin_rating_cnt_7d,
        "parent_asin_rating_avg_prev_rating_7d": parent_asin_rating_avg_prev_rating_7d,
    }
)
train_item_features = item_metadata_pipeline.transform(train_df).astype(np.float32)
sbert_vectors = ann_index.get_vector_by_ids(train_df['item_indice'].values.tolist()).astype(np.float32)
train_item_features = np.hstack([train_item_features, sbert_vectors])

n_users = len(set(user_indices))
n_items = len(set(item_indices))
item_feature_size = train_item_features.shape[1]

model = init_model(n_users, n_items, embedding_dim, item_feature_size, args.dropout)

# Example forward pass
model.eval()
users = torch.tensor(user_indices)
items = torch.tensor(item_indices)
item_sequences = torch.tensor(item_sequences)
item_features = torch.tensor(train_item_features)
predictions = model.predict(users, item_sequences, item_features, items)
print(predictions)

  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[0.4374],
        [0.4554],
        [0.3982],
        [0.4192],
        [0.4361]], grad_fn=<SigmoidBackward0>)


In [13]:
rating_dataset = UserItemBinaryDFDataset(
    train_df,
    "user_indice",
    "item_indice",
    args.rating_col,
    args.timestamp_col,
    item_feature=train_item_features,
)

train_loader = DataLoader(rating_dataset, batch_size=batch_size, shuffle=False)

In [14]:
for batch_input in train_loader:
    print(batch_input)

{'user': tensor([0, 0]), 'item': tensor([0, 1]), 'rating': tensor([0., 1.]), 'item_sequence': tensor([[-1, -1,  2,  3],
        [-1, -1,  2,  3]]), 'item_feature': tensor([[-1.4698e-02,  5.6424e+00, -1.4698e-02,  ...,  2.9136e-02,
         -3.0617e-02,  1.4856e-03],
        [-1.4698e-02, -1.7723e-01, -1.4698e-02,  ..., -2.6627e-03,
         -3.4459e-02,  1.1402e-02]])}
{'user': tensor([1, 2]), 'item': tensor([2, 3]), 'rating': tensor([1., 1.]), 'item_sequence': tensor([[-1, -1,  1,  3],
        [-1, -1,  2,  1]]), 'item_feature': tensor([[-1.4698e-02,  5.6424e+00, -1.4698e-02,  ...,  2.4071e-03,
         -4.1083e-02,  4.6736e-04],
        [-1.4698e-02, -1.7723e-01, -1.4698e-02,  ..., -5.8320e-03,
         -6.7804e-02,  4.7192e-03]])}
{'user': tensor([2]), 'item': tensor([4]), 'rating': tensor([0.]), 'item_sequence': tensor([[-1, -1,  2,  1]]), 'item_feature': tensor([[-1.4698e-02, -1.7723e-01, -1.4698e-02, -2.5463e-02, -1.4698e-02,
         -1.4698e-02, -2.0788e-02, -8.2101e-02, -2.872

In [15]:
# Prepare all item features for recommendation
all_items_df = train_df.drop_duplicates(subset=["item_indice"])
all_items_indices = all_items_df["item_indice"].values
all_items_features = item_metadata_pipeline.transform(all_items_df).astype(np.float32)
all_sbert_vectors = ann_index.get_vector_by_ids(all_items_indices.tolist()).astype(np.float32)
all_items_features = np.hstack([all_items_features, all_sbert_vectors])

lit_model = LitRanker(
    model,
    log_dir=args.notebook_persist_dp,
    all_items_indices=all_items_indices,
    all_items_features=all_items_features
)

# train model
trainer = L.Trainer(
    default_root_dir=f"{args.notebook_persist_dp}/test",
    max_epochs=2,
    accelerator=args.device if args.device else "auto",
)
trainer.fit(
    model=lit_model, train_dataloaders=train_loader, val_dataloaders=train_loader
)

  0%|          | 0/1 [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type   | Params | Mode
----------------------------------------
0 | model | Ranker | 8.2 K  | eval
----------------------------------------
8.2 K     Trainable params
0         Non-trainable params
8.2 K     Total params
0.033     Total estimated model params size (MB)
0         Modules in train mode
14        Modules in eval mode


Sanity Checking: |                                                                                            …

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.
/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.
/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                                                                                                   …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=2` reached.
2024-10-27 22:42:39.368 | INFO     | src.ranker.trainer:on_fit_end:134 - Logging classification metrics...
/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and

In [16]:
# After fitting
model.eval()
predictions = model.predict(users, item_sequences, item_features, items)
print(predictions)

tensor([[0.4297],
        [0.4749],
        [0.4110],
        [0.4287],
        [0.4394]], grad_fn=<SigmoidBackward0>)


In [17]:
# Get the last row of each item as input for recommendations (containing the most updated item_sequence)
to_rec_df = train_df.sort_values(args.timestamp_col, ascending=False).drop_duplicates(
    subset=["user_indice"]
)
recommendations = model.recommend(
    torch.tensor(to_rec_df["user_indice"].values.tolist()),
    torch.tensor(to_rec_df["item_sequence"].values.tolist()),
    torch.tensor(lit_model.all_items_features),
    torch.tensor(lit_model.all_items_indices),
    k=2,
    batch_size=4,
)
recommendations

Generating recommendations:   0%|          | 0/1 [00:00<?, ?it/s]

{'user_indice': [2, 2, 1, 1, 0, 0],
 'recommendation': [2, 4, 0, 2, 1, 4],
 'score': [0.44280707836151123,
  0.43935778737068176,
  0.4133801758289337,
  0.41103968024253845,
  0.47488585114479065,
  0.474662721157074]}

# Prep data

In [18]:
train_df = pd.read_parquet("../data/train_features_neg_df.parquet")
val_df = pd.read_parquet("../data/val_features_neg_df.parquet")
idm_fp = "../data/idm.json"
idm = IDMapper().load(idm_fp)

assert (
    train_df[args.user_col].map(lambda s: idm.get_user_index(s))
    != train_df["user_indice"]
).sum() == 0, "Mismatch IDM"
assert (
    val_df[args.user_col].map(lambda s: idm.get_user_index(s)) != val_df["user_indice"]
).sum() == 0, "Mismatch IDM"

In [19]:
train_df

,user_id,parent_asin,rating,timestamp,parent_asin_rating_cnt_365d,parent_asin_rating_avg_prev_rating_365d,parent_asin_rating_cnt_90d,parent_asin_rating_avg_prev_rating_90d,parent_asin_rating_cnt_30d,parent_asin_rating_avg_prev_rating_30d,...,item_indice,main_category,title,description,categories,price,user_rating_cnt_90d,user_rating_avg_prev_rating_90d,user_rating_list_10_recent_asin,item_sequence
0,AEKSUPM7CH53J3G5PA3JLWLJXUMQ,B00QXJFDZO,0.0,2017-10-30 14:23:22.389,6.0,3.666667,0.0,NaN,0.0,NaN,...,2919,Video Games,Godzilla - PlayStation 4,"[Godzilla, the King of the Monsters, first app...","[Video Games, PlayStation 4, Games]",None,2,5.000000,"B005FVBYV8,B003FMTZSI,B01MS6WG9S,B073W2T5F6","[-1, -1, -1, -1, -1, -1, 1912, 1470, 3498, 3723]"
1,AHSNMFN6DUFTNEZAXBVPIYMXWIFQ,B075MYT126,0.0,2017-11-27 22:01:33.258,NaN,NaN,NaN,NaN,NaN,NaN,...,3777,Video Games,Nintendo Switch Pro Controller - Xenoblade Chr...,[Kick your game sessions up a notch with the N...,"[Video Games, Nintendo Switch, Accessories, Co...",94.98,2,4.000000,"B00CJ9OTNE,B0118YZG0A,B008M502H6,B003Y70W4U,B0...","[2391, 3100, 2176, 1588, 3161, 2133, 2906, 166..."
2,AFE47G5MX35LSHZHZXRYEJFMYPUA,B007VYW5K6,0.0,2017-03-23 21:41:18.000,12.0,4.166667,2.0,4.000000,0.0,NaN,...,2086,Video Games,Crysis 3 [Instant Access],[The award-winning developer crytek is back wi...,"[Video Games, PC]",None,1,NaN,"B07YBX8RNF,B0166QDJDQ,B01CHU4IY4,B00Z9LUDX4,B0...","[4278, 3183, 3288, 3038, 4508, 3391, 3403, 368..."
3,AFJDWGBE3MGULXTO3FUZ5YB6FKDA,B07L5FKGQH,0.0,2017-01-18 15:50:12.000,NaN,NaN,NaN,NaN,NaN,NaN,...,4048,Video Games,Far Cry New Dawn - Deluxe Edition - Xbox [Digi...,"[Dive into a transformed, post-apocalyptic Hop...","[Video Games, Xbox One, Games]",49.88,25,4.416667,"B00I6E6SH6,B00O65I2VY,B005GISQQG,B00008KTNW,B0...","[2632, 2859, 1920, 253, 1428, 1053, 584, 732, ..."
4,AHFDYGJR3SM2D463ZWKGHJPNBKDA,B002BSA2LQ,0.0,2014-01-29 22:50:20.000,2.0,4.500000,1.0,5.000000,0.0,NaN,...,1215,Video Games,Forza Motorsport 3 - Xbox 360,"[Product Description, Whether it’s an exotic s...","[Video Games, Legacy Systems, Xbox Systems, Xb...",31.49,4,5.000000,"B002I0K956,B008CZN458,B0050SXVK8","[-1, -1, -1, -1, -1, -1, -1, 1328, 2146, 1843]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657187,AE5TQ7DBEX2L5T665M6ZDPGYZ32Q,B01LDUYTYS,0.0,2013-10-05 20:20:52.000,NaN,NaN,NaN,NaN,NaN,NaN,...,3442,Video Games,Poochy & Yoshi's Woolly World + Yarn Poochy am...,[Discover a handicraft world bursting at the s...,"[Video Games, Legacy Systems, Nintendo Systems...",249.99,1,NaN,B07X1HF3V6,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 4237]"
657188,AFDG3CXM4DP7X436YNOKTJHVKJQA,B087NNPYP3,5.0,2018-07-10 21:22:10.594,127.0,4.708661,20.0,4.350000,9.0,4.0,...,4342,Video Games,The Legend of Zelda: Breath of the Wild Master...,"[Woken from a 100-year slumber, Link must expl...","[Video Games, Nintendo Switch, Consoles]",None,3,5.000000,"B002I0H79C,B00503E9FY,B00KVOVBGM,B00SHXKC8M,B0...","[1292, 1807, 2734, 2940, 3402, 2759, 2702, 104..."
657189,AFOUC3S3RH7AXMPZBZHLO4WMLLVA,B004AM65C6,0.0,2018-12-16 13:39:37.174,1.0,5.000000,0.0,NaN,0.0,NaN,...,1651,Video Games,Minute to Win It (Kinect) - Xbox 360,"[Product Description, Tackle unique challenges...","[Video Games, Legacy Systems, Xbox Systems, Xb...",12.48,2,5.000000,"B002BSA388,B00PIEI1DG,B08MBHYJP4,B071GPJVTQ,B0...","[1216, 2898, 4397, 3643, 3642, 3527, 3423, 367..."
657190,AEPOGF2QMAXO4W3TYP27DCQRITGA,B07X1HF3V6,0.0,2013-05-30 22:53:17.000,24.0,4.875000,12.0,4.833333,1.0,5.0,...,4237,Video Games,WB Games Mortal Kombat: Komplete Edition - Pla...,[Note:The extra downloadable content is a bonu...,"[Video Games, Legacy Systems, PlayStation Syst...",34.43,4,3.666667,"B0013OL0BK,B002D2Y3IS,B0044R8X9U,B07VLCRZ21,B0...","[-1, -1, -1, -1, -1, 652, 1240, 1629, 4207, 1707]"


In [20]:
user_indices = train_df["user_indice"].unique()
item_indices = train_df["item_indice"].unique()
all_sbert_vectors = ann_index.get_vector_by_ids(item_indices.tolist(), chunk_size=1000).astype(np.float32)

train_item_features = chunk_transform(
    train_df, item_metadata_pipeline, chunk_size=args.tfm_chunk_size
)
train_item_features = train_item_features.astype(np.float32)
train_sbert_vectors = all_sbert_vectors[train_df['item_indice'].values]
train_item_features = np.hstack([train_item_features, train_sbert_vectors])

val_item_features = chunk_transform(
    val_df, item_metadata_pipeline, chunk_size=args.tfm_chunk_size
)
val_item_features = val_item_features.astype(np.float32)
val_sbert_vectors = all_sbert_vectors[val_df['item_indice'].values]
val_item_features = np.hstack([val_item_features, val_sbert_vectors])

logger.info(f"{len(user_indices)=:,.0f}, {len(item_indices)=:,.0f}")

  0%|          | 0/5 [00:00<?, ?it/s]

Transforming chunks:   0%|          | 0/66 [00:00<?, ?it/s]

Transforming chunks:   0%|          | 0/1 [00:00<?, ?it/s]

2024-10-27 22:42:50.271 | INFO     | __main__:<module>:19 - len(user_indices)=19,578, len(item_indices)=4,630


# Train

In [21]:
rating_dataset = UserItemBinaryDFDataset(
    train_df,
    "user_indice",
    "item_indice",
    args.rating_col,
    args.timestamp_col,
    item_feature=train_item_features,
)
val_rating_dataset = UserItemBinaryDFDataset(
    val_df,
    "user_indice",
    "item_indice",
    args.rating_col,
    args.timestamp_col,
    item_feature=val_item_features,
)

train_loader = DataLoader(
    rating_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True
)
val_loader = DataLoader(
    val_rating_dataset, batch_size=args.batch_size, shuffle=False, drop_last=False
)

In [22]:
n_items = len(item_indices)
n_users = len(user_indices)

model = init_model(
    n_users, n_items, args.embedding_dim, item_feature_size, args.dropout
)
model

Ranker(
  (item_embedding): Embedding(4631, 128, padding_idx=4630)
  (user_embedding): Embedding(19578, 128)
  (gru): GRU(128, 128, batch_first=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (item_feature_tower): Sequential(
    (0): Linear(in_features=929, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (fc_rating): Sequential(
    (0): Linear(in_features=512, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

#### Predict before train

In [23]:
val_df = val_rating_dataset.df
val_df.sample(10)

,user_id,parent_asin,rating,timestamp,parent_asin_rating_cnt_365d,parent_asin_rating_avg_prev_rating_365d,parent_asin_rating_cnt_90d,parent_asin_rating_avg_prev_rating_90d,parent_asin_rating_cnt_30d,parent_asin_rating_avg_prev_rating_30d,...,item_indice,main_category,title,description,categories,price,user_rating_cnt_90d,user_rating_avg_prev_rating_90d,user_rating_list_10_recent_asin,item_sequence
3503,AF3GQJW534GEP3I4WKRGKXHBKE3Q,B00DJRLDMU,0.0,2021-12-27 03:58:04.746,0.0,NaN,0.0,NaN,0.0,NaN,...,2448,Video Games,Tom Clancy’s The Division Underground - Xbox O...,"[In Expansion I, Underground, the factions of ...","[Video Games, Xbox One, Downloadable Content]",None,1,NaN,"B00DDILSBG,B01GY35HKE,B072C3VM5F,B01M6C7YE8,B0...","[-1, -1, -1, -1, -1, 2441, 3373, 3691, 3479, 4..."
777,AH2NODYUKDSKO36GWENCKJCA26CQ,B0002XL3BA,0.0,2022-04-05 18:58:50.338,1.0,5.000000,1.0,5.000000,0.0,NaN,...,342,Video Games,God of War - PlayStation 2,[Unleash the power of the Gods and embark on a...,"[Video Games, Legacy Systems, PlayStation Syst...",17.98,1,NaN,"B003DOSRSW,B002JTX78M,B01GY35YXO,B01GKFPFZS,B0...","[-1, -1, -1, -1, 1459, 1346, 3378, 3345, 2581,..."
2429,AGIUOPGPDP4YXZFQPSUPH4WDSP2A,B08SBWDBDB,1.0,2021-10-29 18:03:04.248,1.0,5.000000,0.0,NaN,0.0,NaN,...,4431,Video Games,eXtremeRate Green Purple Chameleon Replacement...,[],"[Video Games, Nintendo Switch, Accessories]",13.99,1,NaN,"B07L3D7C21,B08D3XL1KF,B0719W8QL3,B07CV6LH3V,B0...","[-1, -1, -1, 4043, 4368, 3632, 3908, 3729, 417..."
3558,AHEAWXZ7P75OLPT25RKZYPW4HKSQ,B001EYUSJ4,0.0,2021-12-07 16:23:39.578,1.0,4.000000,0.0,NaN,0.0,NaN,...,955,Video Games,Final Fantasy XIII - Playstation 3,"[Product Description, Final Fantasy XIII is th...","[Video Games, Legacy Systems, PlayStation Syst...",17.97,1,NaN,"B001EYUS7G,B00BQVXUOA,B00KIFM28A,B001SGZL2W,B0...","[949, 2356, 2705, 1149, 2721, 2448, 4284, 3357..."
3013,AGULGCFB6EZHH6LDKKEYR4QQ477Q,B08XD54VJY,1.0,2021-08-16 01:54:55.683,12.0,4.083333,12.0,4.083333,11.0,4.181818,...,4440,Video Games,The Legend of Zelda: Skyward Sword HD - Ninten...,"[Solve puzzles, explore dungeons, and soar the...","[Video Games, Nintendo Switch, Games]",48.49,1,NaN,"B002BSH3K4,B0BWD2QV4N,B001ELJF8M,B07J2KVXY3,B0...","[-1, -1, -1, -1, -1, 1224, 4586, 796, 4016, 531]"
3774,AF5SJBTOYDT5LR7HPQ6OZ77CO3NA,B008HPAXZ2,0.0,2022-03-28 21:57:29.641,0.0,NaN,0.0,NaN,0.0,NaN,...,2157,Video Games,HORI Nintendo 3DS XL Screen Protective Filter,[Officially licensed by Nintendo. This is the ...,"[Video Games, Legacy Systems, Nintendo Systems...",None,1,NaN,"B00000K1VA,B00001XDUB,B00004WMVL,B00000K1VE,B0...","[27, 37, 113, 28, 231, 427, 551, 2321, 4013, 4..."
487,AHCFZPQLIV7236UBXSGU3I5SMXBQ,B00QO4NAOO,1.0,2021-12-07 21:14:02.259,2.0,5.000000,0.0,NaN,0.0,NaN,...,2916,Video Games,MLB 15: The Show (10th Anniversary Edition),[The 10th edition of “The Show” and the best w...,"[Video Games, PlayStation 4]",41.99,3,4.5,"B072V478NR,B076GXJNDZ,B0C5HQ6L7F,B072C3VM5F,B0...","[3705, 3798, 4612, 3691, 3087, 3479, 3723, 391..."
3834,AGEWEMFAC5HSC2DQF4PRJU3UISVA,B00K308KF4,0.0,2022-01-16 20:31:55.061,4.0,3.500000,0.0,NaN,0.0,NaN,...,2694,Video Games,Call of Duty Advanced Warfare - Day Zero Edition,[Order and get access to the Call of Duty: Adv...,"[Video Games, Xbox One, Games]",29.47,1,NaN,"B001EYUU2O,B00K5HTPR2,B07K3KHFSY,B00GN67PJ4,B0...","[-1, 977, 2697, 4028, 2578, 2391, 3007, 3077, ..."
2401,AGJFKYY6UEHVJNRG2IBKLWYHOMUQ,B0080CAO9C,0.0,2022-06-06 03:26:47.178,0.0,NaN,0.0,NaN,0.0,NaN,...,2101,Video Games,Pro Evolution Soccer 2013 - Xbox 360,"[Product Description, This fall Pro Evolution ...","[Video Games, Legacy Systems, Xbox Systems, Xb...",20.25,1,NaN,"B00267S2A0,B001G7PRLS,B003O6E620,B078FBVJMB,B0...","[-1, -1, -1, -1, -1, 1178, 1070, 1511, 3829, 3..."
1059,AEEFFIJ2GOSCLNBGCIDOF7HKXB7Q,B000E36ONK,0.0,2022-06-22 10:20:48.475,0.0,NaN,0.0,NaN,0.0,NaN,...,434,Video Games,Harvest Moon Magical Melody - Gamecube,[Harvest Moon: Magical Melody is a unique new ...,"[Video Games, Legacy Systems, Nintendo S

In [24]:
user_id = val_df.sample(1)[args.user_col].values[0]
# user_id = "AH4AOFTTDPHPAFAAVFMAF25H2LIQ"
test_df = val_df.loc[lambda df: df[args.user_col].eq(user_id)]
with pd.option_context("display.max_colwidth", None):
    display(test_df)

user_id parent_asin  rating  \
1883  AFNGUKYWUWCH6HAANP4DCND6FETA  B001EYUQ8M     0.0   
2965  AFNGUKYWUWCH6HAANP4DCND6FETA  B001E27DLM     0.0   
3420  AFNGUKYWUWCH6HAANP4DCND6FETA  B004WMM2NA     0.0   
3638  AFNGUKYWUWCH6HAANP4DCND6FETA  B079FPFV3X     1.0   

                   timestamp  parent_asin_rating_cnt_365d  \
1883 2022-03-09 13:29:26.011                          1.0   
2965 2022-03-09 13:29:26.011                          4.0   
3420 2022-03-09 13:29:26.011                          1.0   
3638 2022-03-09 13:29:26.011                          7.0   

      parent_asin_rating_avg_prev_rating_365d  parent_asin_rating_cnt_90d  \
1883                                 5.000000                         1.0   
2965                                 5.000000                         0.0   
3420                                 5.000000                         1.0   
3638                                 4.285714                         1.0   

      parent_asin_rating_avg_prev_rating_90d  parent_asin_rating_cnt_30d  \
1883                                     5.0                         0.0   
2965                                     NaN                         0.0   
3420                                     5.0                         1.0   
3638                                     5.0                         1.0   

      parent_asin_rating_avg_prev_rating_30d  ...  item_indice  \
1883                                     NaN  ...          902   
2965                                     NaN  ...          714   
3420                                     5.0  ...         1792   
3638                                     5.0  ...         3851   

        main_category  \
1883      Video Games   
2965             None   
3420      Video Games   
3638  All Electronics   

                                                                                                                                                                   title  \
1883                                                                                                                                BioShock 2 Special Edition -Xbox 360   
2965                                                                                                                                                      Chrono Trigger   
3420                                                                                                                       Animal Crossing: City Folk (Nintendo Selects)   
3638  OIVO PS4 Stand Cooling Fan Station for Playstation 4/PS4 Slim/PS4 Pro, PS4 Pro Vertical Stand with Dual Controller EXT Port Charger Dock Station and 12 Game Slots   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [25]:
test_row = test_df.loc[lambda df: df[args.rating_col].gt(0)].iloc[0]
item_id = test_row[args.item_col]
item_sequence = test_row["item_sequence"]
row_idx = test_row.name
item_feature = val_item_features[row_idx]
logger.info(
    f"Test predicting before training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
user_indice = idm.get_user_index(user_id)
item_indice = idm.get_item_index(item_id)
user = torch.tensor([user_indice])
item_sequence = torch.tensor([item_sequence])
item_feature = torch.tensor([item_feature])
item = torch.tensor([item_indice])

model.eval()
model.predict(user, item_sequence, item_feature, item)

2024-10-27 22:42:52.873 | INFO     | __main__:<module>:6 - Test predicting before training with user_id = AFNGUKYWUWCH6HAANP4DCND6FETA and parent_asin = B079FPFV3X
/var/folders/mq/xtxzhvlj4m97tb8lgwxcnsb40000gn/T/ipykernel_77639/2733117192.py:12: UserWarning:

Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)



tensor([[0.4902]], grad_fn=<SigmoidBackward0>)

#### Training loop

##### Overfit 1 batch

In [26]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=10, mode="min", verbose=False
)

model = init_model(n_users, n_items, args.embedding_dim, item_feature_size, dropout=0)
lit_model = LitRanker(
    model,
    learning_rate=args.learning_rate,
    l2_reg=0.0,
    log_dir=args.notebook_persist_dp,
)

log_dir = f"{args.notebook_persist_dp}/logs/overfit"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    accelerator=args.device if args.device else "auto",
    max_epochs=100,
    overfit_batches=1,
    callbacks=[early_stopping],
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=train_loader,
)
logger.info(f"Logs available at {trainer.log_dir}")

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.

  | Name  | Type   | Params | Mode 
-----------------------------------------
0 | model | Ranker | 3.4 M  | train
-----------------------------------------
3.4 M     Trainable params
0         Non-trainable params
3.4 M     Total params
13.533    Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                            …

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:251: UserWarning:

You requested to overfit but enabled val dataloader shuffling. We are turning off the val dataloader shuffling for you.

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:251: UserWarning:

You requested to overfit but enabled train dataloader shuffling. We are turning off the train dataloader shuffling for you.

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_conne

Training: |                                                                                                   …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=100` reached.
2024-10-27 22:43:10.183 | INFO     | src.ranker.trainer:on_fit_end:134 - Logging classification metrics...
2024-10-27 22:44:00.625 | INFO     | __main__:<module>:28 - Logs available at /Users/dvq/frostmourne/recsys-mvp/notebooks/data/017-item-popularity/logs/overfit/lightning_logs/version_5


In [27]:
%tensorboard --logdir $trainer.log_dir

##### Fit on all data

In [28]:
all_items_df = train_df.drop_duplicates(subset=["item_indice"])
all_items_indices = all_items_df["item_indice"].values
all_items_features = item_metadata_pipeline.transform(all_items_df).astype(np.float32)
all_sbert_vectors = ann_index.get_vector_by_ids(all_items_indices.tolist()).astype(np.float32)
all_items_features = np.hstack([all_items_features, all_sbert_vectors])

  0%|          | 0/47 [00:00<?, ?it/s]

In [29]:
len(all_items_indices)

4630

In [30]:
all_items_features.shape

(4630, 929)

In [31]:
# papermill_description=fit-model
early_stopping = EarlyStopping(
    monitor="val_loss", patience=args.early_stopping_patience, mode="min", verbose=False
)

checkpoint_callback = ModelCheckpoint(
    dirpath=f"{args.notebook_persist_dp}/checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    monitor="val_loss",
    mode="min",
)

model = init_model(
    n_users,
    n_items,
    args.embedding_dim,
    item_feature_size,
    dropout=args.dropout,
    item_embedding=pretrained_item_embedding,
)
lit_model = LitRanker(
    model,
    learning_rate=args.learning_rate,
    l2_reg=args.l2_reg,
    log_dir=args.notebook_persist_dp,
    evaluate_ranking=True,
    idm=idm,
    all_items_indices=all_items_indices,
    all_items_features=all_items_features,
    args=args,
    neg_to_pos_ratio=args.neg_to_pos_ratio,
)

log_dir = f"{args.notebook_persist_dp}/logs/run"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    max_epochs=args.max_epochs,
    callbacks=[early_stopping, checkpoint_callback],
    accelerator=args.device if args.device else "auto",
    logger=args._mlf_logger if args.log_to_mlflow else None,
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: UserWarning:

Checkpoint directory /Users/dvq/frostmourne/recsys-mvp/notebooks/data/017-item-popularity/checkpoints exists and is not empty.


  | Name  | Type   | Params | Mode 
-----------------------------------------
0 | model | Ranker | 3.4 M  | train
-----------------------------------------
3.4 M     Trainable params
0         Non-trainable params
3.4 M     Total params
13.533    Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                            …

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: PossibleUserWarning:

The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.



Training: |                                                                                                   …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

2024-10-27 23:08:39.430 | INFO     | src.ranker.trainer:on_fit_end:134 - Logging classification metrics...
2024-10-27 23:08:40.540 | INFO     | src.ranker.trainer:on_fit_end:137 - Logging ranking metrics...


Generating recommendations:   0%|          | 0/177 [00:00<?, ?it/s]

2024/10/27 23:09:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run 017-item-popularity at: http://localhost:5002/#/experiments/3/runs/c5a5c02ddd594693b90788abae264d36.
2024/10/27 23:09:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5002/#/experiments/3.


In [32]:
logger.info(
    f"Test predicting after training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
model.eval()
model.predict(user, item_sequence, item_feature, item)

2024-10-27 23:09:24.361 | INFO     | __main__:<module>:1 - Test predicting after training with user_id = AFNGUKYWUWCH6HAANP4DCND6FETA and parent_asin = B079FPFV3X


tensor([[0.8907]], grad_fn=<SigmoidBackward0>)

# Load best checkpoint

In [33]:
logger.info(f"Loading best checkpoint from {checkpoint_callback.best_model_path}...")
args.best_checkpoint_path = checkpoint_callback.best_model_path

best_trainer = LitRanker.load_from_checkpoint(
    checkpoint_callback.best_model_path,
    model=init_model(
        n_users, n_items, args.embedding_dim, item_feature_size, dropout=0
    ),
)

2024-10-27 23:09:24.421 | INFO     | __main__:<module>:1 - Loading best checkpoint from /Users/dvq/frostmourne/recsys-mvp/notebooks/data/017-item-popularity/checkpoints/best-checkpoint-v5.ckpt...


In [34]:
best_model = best_trainer.model.to(lit_model.device)

In [35]:
best_model.eval()
best_model.predict(user, item_sequence, item_feature, item)

tensor([[0.7696]], grad_fn=<SigmoidBackward0>)

### Persist id mapping

In [36]:
if args.log_to_mlflow:
    # Persist id_mapping so that at inference we can predict based on item_ids (string) instead of item_index
    run_id = trainer.logger.run_id
    mlf_client = trainer.logger.experiment
    mlf_client.log_artifact(run_id, idm_fp)
    # Persist item_feature_metadata pipeline
    mlf_client.log_artifact(run_id, args.item_metadata_pipeline_fp)

### Wrap inference function and register best checkpoint as MLflow model

In [37]:
inferrer = RankerInferenceWrapper(best_model)

In [47]:
sample_input = {
    "user_ids": [idm.get_user_id(0)],
    "item_sequences": [[idm.get_item_id(0), idm.get_item_id(1)]],
    **{col: [train_df.iloc[0].fillna(0)[col]] for col in args.item_feature_cols},
    "item_ids": [idm.get_item_id(0)],
}
sample_output = inferrer.infer([0], [[0, 1]], [train_item_features[0]], [0])
sample_output

array([0.63961154], dtype=float32)

In [48]:
sample_input

{'user_ids': ['AE225O22SA7DLBOGOEIFL7FT5VYQ'],
 'item_sequences': [['0375869026', '9625990674']],
 'main_category': ['Video Games'],
 'categories': [array(['Video Games', 'PlayStation 4', 'Games'], dtype=object)],
 'price': ['None'],
 'parent_asin_rating_cnt_365d': [6.0],
 'parent_asin_rating_avg_prev_rating_365d': [3.6666666666666665],
 'parent_asin_rating_cnt_90d': [0.0],
 'parent_asin_rating_avg_prev_rating_90d': [0],
 'parent_asin_rating_cnt_30d': [0.0],
 'parent_asin_rating_avg_prev_rating_30d': [0],
 'parent_asin_rating_cnt_7d': [0.0],
 'parent_asin_rating_avg_prev_rating_7d': [0],
 'item_ids': ['0375869026']}

In [49]:
if args.log_to_mlflow:
    run_id = trainer.logger.run_id
    sample_output_np = sample_output
    signature = infer_signature(sample_input, sample_output_np)
    idm_filename = idm_fp.split("/")[-1]
    item_metadata_pipeline_filename = args.item_metadata_pipeline_fp.split("/")[-1]
    with mlflow.start_run(run_id=run_id):
        mlflow.pyfunc.log_model(
            python_model=inferrer,
            artifact_path="inferrer",
            # We log the id_mapping to the predict function so that it can accept item_id and automatically convert ot item_indice for PyTorch model to use
            artifacts={
                "idm": mlflow.get_artifact_uri(idm_filename),
                "item_metadata_pipeline": mlflow.get_artifact_uri(
                    item_metadata_pipeline_filename
                ),
            },
            signature=signature,
            input_example=sample_input,
            registered_model_name=args.mlf_model_name,
        )

/Users/dvq/frostmourne/recsys-mvp/.venv/lib/python3.11/site-packages/mlflow/models/utils.py:523: FutureWarning:

Since MLflow 2.16.0, we no longer convert dictionary input example to pandas Dataframe, and directly save it as a json object. If the model expects a pandas DataFrame input instead, please pass the pandas DataFrame as input example directly.



Registered model 'ranker' already exists. Creating a new version of this model...
2024/10/27 23:18:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ranker, version 15
Created version '15' of model 'ranker'.


2024/10/27 23:18:22 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": {
    "user_ids": [
      "AE225O22SA7DLBOGOEIFL7FT5VYQ"
    ],
    "item_sequences": [
      [
        "0375869026",
        "9625990674"
      ]
    ],
    "main_category": [
      "Video Games"
    ],
    "categories": [
      [
        "Video Games",
        "PlayStation 4",
        "Games"
      ]
    ],
    "price": [
      "None"
    ],
    "parent_asin_rating_cnt_365d": [
      6.0
    ],
    "parent_asin_rating_avg_prev_rating_365d": [
      3.6666666666666665
    ],
    "parent_asin_rating_cnt_90d": [
      0.0
    ],
    "parent_asin_rating_avg_prev_rating_90d": [
      0
    ],
    "parent_asin_rating_cnt_30d": [
      0.0
    ],
    "parent_asin_rating_avg_prev_rating_30d": [
      0
    ],
    "parent_asin_rating_cnt_7d": [
      0.0
    ],
    "parent_asin_rating_avg_prev_rating_7d": [
      0
    ],
    "item_ids": [
      "0375869026"
    ]
  }
}. Alternatively, you 

# Set the newly trained model as champion

In [50]:
if args.log_to_mlflow:
    val_roc_auc = trainer.logger.experiment.get_run(trainer.logger.run_id).data.metrics[
        "val_roc_auc"
    ]

    if val_roc_auc > args.min_roc_auc:
        logger.info(f"Aliasing the new model as champion...")
        model_version = (
            mlf_client.get_registered_model(args.mlf_model_name)
            .latest_versions[0]
            .version
        )

        mlf_client.set_registered_model_alias(
            name=args.mlf_model_name, alias="champion", version=model_version
        )

        mlf_client.set_model_version_tag(
            name=args.mlf_model_name,
            version=model_version,
            key="author",
            value="quy.dinh",
        )

2024-10-27 23:18:27.661 | INFO     | __main__:<module>:7 - Aliasing the new model as champion...


# Clean up

In [51]:
all_params = [args]

if args.log_to_mlflow:
    with mlflow.start_run(run_id=run_id):
        for params in all_params:
            params_dict = params.dict()
            params_ = dict()
            for k, v in params_dict.items():
                if k == "top_K":
                    k = "top_big_K"
                if k == "top_k":
                    k = "top_small_k"
                params_[f"{params.__repr_name__()}.{k}"] = v
            mlflow.log_params(params_)

2024/10/27 23:18:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run 017-item-popularity at: http://localhost:5002/#/experiments/3/runs/c5a5c02ddd594693b90788abae264d36.
2024/10/27 23:18:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5002/#/experiments/3.
